In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
commits = pd.read_feather('../data/commits.feather')
commits.head()

,commit_id,project_id,project_name,parent_id
0,321132855,26730207,meta-ivi,321132856.0
1,333294039,27617768,ExcelLaunchPad,333294036.0
2,302312189,24513310,yii2-webception,302312191.0
3,124854783,3452681,mammoth.js,124843456.0
4,423307958,32130467,phase-0,423307476.0


In [3]:
project_names_sample = commits[['project_name']].sample(5000, random_state = 2019)
commits_sample = commits.loc[commits['project_name'].isin(project_names_sample.project_name.values)]
commits_sample.head()

,commit_id,project_id,project_name,parent_id
2,302312189,24513310,yii2-webception,302312191.0
5,216345610,15364771,ProjectEuler,216345505.0
10,378012761,31249832,test,378012249.0
11,172463423,10619145,ShaiWaWa,172463425.0
17,286691702,22814549,bbbike-world,286691703.0


In [4]:
class GraphExtracter():
    def __init__(self, graph):
        self.graph = graph
        
    def get_nodes(self):
        self.num_nodes = self.graph.number_of_nodes()
        return self.num_nodes
    
    def get_edges(self):
        self.num_edges = self.graph.number_of_edges()
        return self.num_edges
    
    def get_density(self):
        self.density = nx.density(self.graph)
        return self.density
    
    def get_avg_clustering(self):
        self.avg_clustering = nx.algorithms.cluster.average_clustering(self.graph)
        return self.avg_clustering
    
    def get_transitivity(self):
        self.transitivity = nx.algorithms.cluster.transitivity(self.graph)
        return self.transitivity
    
    def get_weakly_connected(self):
        self.weakly_connected = nx.algorithms.components.is_weakly_connected(self.graph)
        return self.weakly_connected
    
    def get_num_weakly_connected(self):
        self.num_weakly_connected = nx.algorithms.components.number_weakly_connected_components(self.graph)
        return self.num_weakly_connected
    
    def get_num_attrac_components(self):
        self.num_attrac_components = nx.algorithms.components.number_attracting_components(self.graph)
        return self.num_attrac_components
    
    def get_avg_degree(self):
        degree = nx.degree(self.graph)
        degree = list(degree)
        degree = [entry[1] for entry in degree]
        self.avg_degree = np.mean(degree)
        return self.avg_degree
    
    def get_avg_degree_centrality(self):
        self.avg_degree_centrality = np.mean(list(nx.algorithms.centrality.degree_centrality(self.graph).values()))
        return self.avg_degree_centrality
    
    def get_avg_in_degree(self):
        in_ = self.graph.in_degree
        in_ = list(in_)
        in_ = [entry[1] for entry in in_]
        self.avg_in_degree = np.mean(in_)
        return self.avg_in_degree
    
    def get_avg_in_degree_centrality(self):
        self.avg_in_degree_centrality = np.mean(list(nx.algorithms.centrality.in_degree_centrality(self.graph).values()))
        return self.avg_in_degree_centrality
    
    def get_avg_out_degree(self):
        out = self.graph.out_degree
        out = list(out)
        out = [entry[1] for entry in out]
        self.avg_out_degree = np.mean(out)
        return self.avg_out_degree
    
    def get_avg_out_degree_centrality(self):
        self.avg_out_degree_centrality = np.mean(list(nx.algorithms.centrality.out_degree_centrality(self.graph).values()))
        return self.avg_out_degree_centrality
    
    def get_eigen_centrality(self):
        self.eigen_centrality = np.mean(list(nx.algorithms.centrality.eigenvector_centrality(self.graph, max_iter=int(1e6)).values()))
        return self.eigen_centrality
    
    def get_katz_centrality(self):
        self.katz_centrality = np.mean(list(nx.algorithms.centrality.katz_centrality(self.graph, max_iter=int(1e6)).values()))
        return self.katz_centrality
    
    def get_num_triangles(self):
        self.num_triangles = np.mean(list(nx.algorithms.cluster.clustering(self.graph).values()))
        return self.num_triangles
    
    def set_all_features(self):
        self.get_nodes()
        self.get_edges()
        self.get_density()
        self.get_avg_clustering()
        self.get_transitivity()
        self.get_weakly_connected()
        self.get_num_weakly_connected()
        self.get_num_attrac_components()
        self.get_avg_degree()
        self.get_avg_degree_centrality()
        self.get_avg_in_degree()
        self.get_avg_in_degree_centrality()
        self.get_avg_out_degree()
        self.get_avg_out_degree_centrality()
        self.get_eigen_centrality()
        self.get_katz_centrality()
        self.get_num_triangles()
        return None

    def get_all_features(self):
        return {
            'num_nodes': self.num_nodes,
            'num_edges': self.num_edges,
            'density': self.density,
            'avg_clustering': self.avg_clustering,
            'transitivity': self.transitivity,
            'weakly_connected': self.weakly_connected,
            'num_weakly_connected': self.num_weakly_connected,
            'num_attrac_components': self.num_attrac_components,
            'avg_degree': self.avg_degree,
            'avg_degree_centrality': self.avg_degree_centrality,
            'avg_in_degree': self.avg_in_degree,
            'avg_in_degree_centrality': self.avg_in_degree_centrality,
            'avg_out_degree': self.avg_out_degree,
            'avg_out_degree_centrality': self.avg_out_degree_centrality,
            'eigen_centrality': self.eigen_centrality,
            'katz_centrality': self.katz_centrality,
            'num_triangles': self.num_triangles}

In [ ]:
graphs = {}

for idx, project in enumerate(project_names_sample.project_name.values):
    if idx%10 == 0:
        print(f'Percentage Completed: {idx/5}%')
    graph_df = commits_sample.loc[commits_sample['project_name'] == project]
    graph_df = graph_df[['commit_id', 'parent_id']]
    graph_df.columns = pd.Index(['target', 'source'])
    
    if graph_df.shape[0] > 5000:
        continue
    
    graph = nx.from_pandas_edgelist(graph_df, create_using=nx.DiGraph)
    graph.name = project
    graph_extracter = GraphExtracter(graph)
    graph_extracter.set_all_features()
    graphs[project] = graph_extracter.get_all_features()
    
    nx.write_gpickle(graph, f'../data/graphs/graph_pickles/{project}.gpickle')

In [ ]:
graph_features = pd.DataFrame(graphs).T

In [ ]:
from sklearn.preprocessing import scale
scaled_graph_features = scale(graph_features)

In [ ]:
for i in range(5, 105):
    if i%10 != 0:
        continue
    X_embedded = TSNE(n_components=2, perplexity=i).fit_transform(scaled_graph_features)
    plt.scatter(X_embedded[:, 0], X_embedded[:, 1])
    plt.title(f't-SNE Perplexity = {i}')
    plt.show()

In [ ]:
inertia = []
for i in range(1, 20):
    kmeans = KMeans(n_clusters=i, random_state=2019).fit(scaled_graph_features)
    inertia.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 20), inertia)
plt.scatter(range(1, 20), inertia);

In [ ]:
kmeans = KMeans(n_clusters = 4, random_state=2019).fit(scaled_graph_features)

for i in range(5, 105):
    if i%10 != 0:
        continue
    X_embedded = TSNE(n_components=2, perplexity=i).fit_transform(scaled_graph_features)
    plt.scatter(X_embedded[:, 0], X_embedded[:, 1], c=kmeans.labels_)
    plt.title(f't-SNE Perplexity = {i}')
    plt.show()

### Features

- Number of Nodes
- Number of Edges
- Density
- Average Clustering Coefficient
- Transitivity
- Weakly Connected
- Number of Weakly Connected
- Number attracting components

- Degree Centrality **dict**
- In-Degree Centrality **dict**
- Out-Degree Centrality **dict**
- Eigenvector Centrality **dict**
- Katz Centrality **dict**
- Current-flow Closeness Centrality **dict**
- Betweenness Centrality **dict**
- Clustering **dict**

- Average degree (overall)
- Average in degree
- Average out degree